# ДОМАШНЕЕ ЗАДАНИЕ 3. Классификация текстовых документов
## Цель работы

Приобрести опыт решения практических задач по машинному обучению, таких как анализ и визуализация исходных данных, обучение, выбор и оценка качества моделей предсказания, посредством языка программирования Python.

## Библиотеки

In [ ]:
import time
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, recall_score, precision_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import (
    TfidfVectorizer,
    CountVectorizer,
    HashingVectorizer,
    TfidfTransformer
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB

# Выключаем ворнинги, чтобы не засорять вывод при обучении логистической регрессии
# с неоптиимальными параметрами (при GridSearchCV такое будет)
warnings.filterwarnings('ignore')

# Фиксируем RANDOM_STATE как константу для воспроизводимости результатов
RANDOM_STATE = 123

## Вариант

In [ ]:
surname = "Панфилкин" # Ваша фамилия

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [4, 42, 21, 21, 34,  1, 44, 26, 18, 43, 38, 26, 18, 43,  3, 49, 45,
        7, 42, 25,  4,  9, 36, 33, 31, 29,  5, 31,  4, 19, 24, 27, 33]
d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 3 + 1
print("Ваш вариант - ", variant)

### Задание 1. Оценка качества классификации текстовых данных (2 балла)

#### Загрузка данных

In [ ]:
# Загрузка данных из файла data/reviews.tsv (первый столбец - метки классов, второй - тексты)
data = pd.read_csv('data/reviews.tsv', sep='\t', names=['target', 'text'])
# Вытащим X и y из датафрейма
X, y = data['text'], data['target']
# Выведем количество наблюдений в каждом классе
for target in y.unique():
    print(f'Класс {target}: {len(y[y == target])} наблюдений')
# Отобразим датафрейм с данными
data

#### Разделение данных

In [ ]:
# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size=0.8,
    random_state=RANDOM_STATE,
)

# Также определим KFold для дальнейшей кросс-валидации (везде будем использовать данный объект)
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)

#### Определение функций

In [ ]:
def get_metrics(pipeline, X_train, y_train, X_test, y_test):
    """
    Функция для расчета метрик в виде словоря
    """
    # Раскроем пайплайн и отдельно обучим векторайзер, чтобы время обучения векторайзера
    # не учитывалось в метриках
    vectorizer = pipeline['vectorizer']
    classifier = pipeline['classifier']

    # Векторизуем
    X_train_ = vectorizer.fit_transform(X_train)
    X_test_ = vectorizer.transform(X_test)

    # Обучаем
    start = time.time()
    classifier.fit(X_train_, y_train)
    fit_time = time.time() - start

    # Предсказываем
    start = time.time()
    y_pred = classifier.predict(X_test_)
    predict_time = time.time() - start

    # Возвращаем все метрики 
    return {
        'fit_time': fit_time,
        'score_time': predict_time,
        'test_balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
        'test_recall': recall_score(y_test, y_pred),
        'test_precision': precision_score(y_test, y_pred),
        'test_f1': f1_score(y_test, y_pred)
    }

def cross_validate(pipeline, X, y, kf):
    """
    Функция для расчета метрик в виде словоря для кросс валидации
    """
    # Тут будем хранить результаты метрик по фолдам
    metrics_list = []
    for train_idx, test_idx in kf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        metrics_list.append(get_metrics(pipeline, X_train, y_train, X_test, y_test))
    # Через pandas усредним и приведем в одинаковый с get_metrics() вид
    return pd.DataFrame(metrics_list).mean().to_dict()

#### Определение моделей

In [ ]:
# Для моделей определим векторайзер TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)
# Для модели Бернулли используем бинарный векторайзер
vectorizer_binary = CountVectorizer(binary=True, ngram_range=(1, 1), lowercase=True)

# Определим пайплайны
pipelines = {
    "К-ближайших соседей": Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', KNeighborsClassifier(n_neighbors=5))
    ]),
    "Логистическая регрессия": Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', LogisticRegression(
            penalty='l2',
            fit_intercept=True,
            max_iter=100,
            C=1,
            solver='lbfgs',
            random_state=RANDOM_STATE
        ))
    ]),
    "Наивный Байес: модель Бернулли": Pipeline([
        ('vectorizer', vectorizer_binary),
        ('classifier', BernoulliNB(alpha=1))
    ]),
    "Наивный Байес: полиномиальная модель": Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', MultinomialNB(alpha=1))
    ])
}
# Для единства храним значения параметров (которые в последствии будем менять)
param_names = ["classifier__n_neighbors", "classifier__C", "classifier__alpha", "classifier__alpha"]
# И их значеня (можно было бы вытащить по имени, но так просто проще сейчас сделать)
param_values = [5, 1, 1, 1]

#### Оценка моделей по отложенной выборке

In [ ]:
metrics_dict = {}
for i, (method_name, pipeline) in enumerate(pipelines.items()):
    param_name = param_names[i]
    param_value = param_values[i]
    label = f"{method_name} ({param_name}={param_value})"

    metrics_dict[label] = get_metrics(pipeline, X_train, y_train, X_test, y_test)

metrics_df = pd.DataFrame(metrics_dict).T
metrics_df

### Задание 2. Оценка качества классификации текстовых данных посредством кросс-валидации (2 балла)

#### Оценка моделей по кросс-валидации

In [ ]:
metrics_dict = {}
for i, (method_name, pipeline) in enumerate(pipelines.items()):
    param_name = param_names[i]
    param_value = param_values[i]
    label = f"{method_name} ({param_name}={param_value})"
    
    metrics_dict[label] = cross_validate(pipeline, X, y, skf)

metrics_df = pd.DataFrame(metrics_dict).T
metrics_df

### Задание 3. Выбор модели (4 баллов)

#### Определение функций

In [ ]:
def get_validation_scores(
        pipelines, # Словарь с пайплайнами {название метода: пайплайн}
        param_names, # Список с названиями параметров (размер как pipelines)
        param_values_list # Список со списками значений параметров (размер как pipelines)
    ):
    '''
    Функция для расчета метрик поиска по сетке в виде словоря используя кросс валидацию.
    '''
    # Словарь для хранения результатов
    validation_scores = {}
    # Для каждого метода
    for i, (method_name, pipeline) in enumerate(pipelines.items()):
        print(f"Поиск лучшего значения гиперпараметра для модели '{method_name}'...")
        # Определяем параметры для поиска по сетке
        gs = GridSearchCV(
            pipeline,
            param_grid={f"{param_names[i]}": param_values_list[i]},
            scoring='balanced_accuracy',
            cv=skf,
            refit=False, # Не обучаем модель на всех данных в конце (Экономим время)
            return_train_score=True, # Возвращаем значения метрики на обучающей выборке
        )
        # Обучаем модель
        gs.fit(X_train, y_train)
        # Вытаскиваем нужные нам метрики и сохраняем в словарь
        validation_scores[method_name] = {
            'train_score': gs.cv_results_['mean_train_score'],
            'test_score': gs.cv_results_['mean_test_score'],
            'fit_time': gs.cv_results_['mean_fit_time'],
            'score_time': gs.cv_results_['mean_score_time'],
            'best_param_index': gs.best_index_
        }
    return validation_scores

def plot_validation_scores(
        validation_scores, # Словарь с метриками из get_validation_scores()
        score_keys, # Список с ключами метрик для отрисовки
        score_labels, # Список с подписями метрик для отрисовки
        highlight_key, # Ключ метрики для подсветки лучшего значения
        param_names, # Список с названиями параметров (размер как pipelines)
        param_values_list, # Список со списками значений параметров (размер как pipelines)
        xscales, # Список с типами шкал для оси X (линейная или логарифмическая)
        title # Значение ngram_range для отрисовки в заголовке
    ):
    '''
    Функция для визуализации результатов поиска по сетке.
    '''
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    axes = axes.flatten()
    for i, (ax, method_name) in enumerate(zip(axes, validation_scores)):
        scores = validation_scores[method_name]
        for score_key, score_label in zip(score_keys, score_labels):
            ax.plot(param_values_list[i], scores[score_key], label=score_label, marker='o')

        if highlight_key is not None:
            # Определение лучшего значения гиперпараметра
            best_param_index = validation_scores[method_name]['best_param_index']
            best_param = param_values_list[i][best_param_index]
            best_accuracy = validation_scores[method_name]['test_score'][best_param_index]
            # Отметим точку с лучшим значением на графике
            ax.scatter(best_param, best_accuracy, marker='o', color='black', zorder=3)
            # Добавим подпись
            ax.annotate(
                f"Top Result:\n({best_param:.2f}, {best_accuracy:.2f})",
                xy=(best_param, best_accuracy),
                xytext=(best_param, best_accuracy - 0.02),
                horizontalalignment='center',
                verticalalignment='top'
            )

        ax.set_title(method_name)
        ax.set_xlabel(param_names[i])
        ax.set_ylabel('Сбалансированная точность')
        ax.set_xscale(xscales[i])
        ax.legend()

    # Фиксим пересечение подписей
    fig.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

def set_best_params(pipelines, param_names, param_values_list, validation_scores):
    for i, (method_name, pipeline) in enumerate(pipelines.items()):
        best_param_index = validation_scores[method_name]['best_param_index']
        best_param = param_values_list[i][best_param_index]
        pipeline.set_params(**{f'{param_names[i]}': best_param})

In [ ]:
# Определим диапазоны значений гиперпараметров
param_values_list = [
    np.arange(1, 150, 20),
    np.logspace(-2, 10, 8, base=10),
    np.logspace(-4, 1, 8, base=10),
    np.logspace(-4, 1, 8, base=10)
]

# Список всех метрик лучгих моделей при каждом значении n-грамм
metrics_list = []
# Список оценок качества моделей при каждом значении n-грамм
validation_scores = {}

#### Оценка влияния гиперпараметров на точность

In [ ]:
ngram = (1, 1)
vectorizer.set_params(ngram_range=ngram)
vectorizer_binary.set_params(ngram_range=ngram)
validation_scores[ngram] = get_validation_scores(pipelines, param_names, param_values_list)

#### График гиперпараметров

In [ ]:
plot_validation_scores(
    validation_scores=validation_scores[ngram],
    score_keys=['train_score', 'test_score'],
    score_labels=['Train', 'Test'],
    highlight_key='test_score',
    param_names=param_names,
    param_values_list=param_values_list,
    xscales=['linear', 'log', 'log', 'log'],
    title=f'Зависимость сбалансированной точности от гиперпараметров моделей [ngram={ngram}]'
)

In [ ]:
set_best_params(pipelines, param_names, param_values_list, validation_scores[ngram])
for i, (method_name, pipeline) in enumerate(pipelines.items()):
    param_name = param_names[i]
    best_param_index = validation_scores[ngram][method_name]['best_param_index']
    best_param = param_values_list[i][best_param_index]
    metrics = cross_validate(pipeline, X, y, skf)
    metrics = {
        'method': method_name,
        'ngram': str(ngram),
        'param': f"{param_name}={round(best_param, 2)}",
        **metrics
    }
    metrics_list.append(metrics)

#### ngram=(2, 2)

In [ ]:
ngram = (2, 2)
vectorizer.set_params(ngram_range=ngram)
vectorizer_binary.set_params(ngram_range=ngram)
validation_scores[ngram] = get_validation_scores(pipelines, param_names, param_values_list)

In [ ]:
plot_validation_scores(
    validation_scores=validation_scores[ngram],
    score_keys=['train_score', 'test_score'],
    score_labels=['Train', 'Test'],
    highlight_key='test_score',
    param_names=param_names,
    param_values_list=param_values_list,
    xscales=['linear', 'log', 'log', 'log'],
    title=f'Зависимость сбалансированной точности от гиперпараметров моделей [ngram={ngram}]'
)

In [ ]:
set_best_params(pipelines, param_names, param_values_list, validation_scores[ngram])
for i, (method_name, pipeline) in enumerate(pipelines.items()):
    param_name = param_names[i]
    best_param_index = validation_scores[ngram][method_name]['best_param_index']
    best_param = param_values_list[i][best_param_index]
    metrics = cross_validate(pipeline, X, y, skf)
    metrics = {
        'method': method_name,
        'ngram': str(ngram),
        'param': f"{param_name}={round(best_param, 2)}",
        **metrics
    }
    metrics_list.append(metrics)

#### ngram=(1, 2)

In [ ]:
ngram = (1, 2)
vectorizer.set_params(ngram_range=ngram)
vectorizer_binary.set_params(ngram_range=ngram)
validation_scores[ngram] = get_validation_scores(pipelines, param_names, param_values_list)

In [ ]:
plot_validation_scores(
    validation_scores=validation_scores[ngram],
    score_keys=['train_score', 'test_score'],
    score_labels=['Train', 'Test'],
    highlight_key='test_score',
    param_names=param_names,
    param_values_list=param_values_list,
    xscales=['linear', 'log', 'log', 'log'],
    title=f'Зависимость сбалансированной точности от гиперпараметров моделей [ngram={ngram}]'
)

In [ ]:
set_best_params(pipelines, param_names, param_values_list, validation_scores[ngram])
for i, (method_name, pipeline) in enumerate(pipelines.items()):
    param_name = param_names[i]
    best_param_index = validation_scores[ngram][method_name]['best_param_index']
    best_param = param_values_list[i][best_param_index]
    metrics = cross_validate(pipeline, X, y, skf)
    metrics = {
        'method': method_name,
        'ngram': str(ngram),
        'param': f"{param_name}={round(best_param, 2)}",
        **metrics
    }
    metrics_list.append(metrics)

In [ ]:
# Сохраним метрики в датафрейм и отобразим как таблицу
metrics_df = pd.DataFrame(metrics_list)
metrics_df

Выводы:

### Задание 4. Оценка влияния количества признаков FeatureHasher на качество классификации (2 баллов)

In [ ]:
hashing_vectorizer = HashingVectorizer(norm=None, alternate_sign=False)
tfidf_transformer = TfidfTransformer(norm=None)

pipelines_hashing = {
    'К-ближайших соседей': Pipeline([
        ('vectorizer', hashing_vectorizer),
        ('transformer', tfidf_transformer),
        ('classifier', KNeighborsClassifier(n_neighbors=5))
    ]),
    'Логистическая регрессия': Pipeline([
        ('vectorizer', hashing_vectorizer),
        ('transformer', tfidf_transformer),
        ('classifier', LogisticRegression(
            penalty='l2',
            fit_intercept=True,
            max_iter=100,
            C=1,
            solver='lbfgs',
            random_state=RANDOM_STATE
        ))
    ]),
    "Наивный Байес: модель Бернулли": Pipeline([
        ('vectorizer', hashing_vectorizer),
        # Трансформер не нужен
        ('classifier', BernoulliNB(alpha=1.0))
    ]),
    "Наивный Байес: полиномиальная модель": Pipeline([
        ('vectorizer', hashing_vectorizer),
        # Трансформер не нужен
        ('classifier', MultinomialNB(alpha=1.0))
    ])
}

# Установим лучшие параметры при ngram=(1, 1)
ngram = (1, 1)
set_best_params(pipelines_hashing, param_names, param_values_list, validation_scores[ngram])

In [ ]:
param_names_hashing = ['vectorizer__n_features']*4
param_values_list_hashing = [np.logspace(1, 5, 5, base=10, dtype=int)]*4

In [ ]:
validation_scores_hashing = get_validation_scores(pipelines_hashing, param_names_hashing, param_values_list_hashing)

In [ ]:
plot_validation_scores(
    validation_scores=validation_scores_hashing,
    score_keys=['train_score', 'test_score'],
    score_labels=['Train', 'Test'],
    highlight_key='test_score',
    param_names=param_names_hashing,
    param_values_list=param_values_list_hashing,
    xscales=['log']*4,
    title=f'Зависимость сбалансированной точности от гиперпараметров моделей [ngram={ngram}]'
)

Выводы: